In [6]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
file_path = '/mnt/d/codes/Datasets/datasetNew.csv'
df = pd.read_csv(file_path)

# Extract resource constraints (last row)
constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values
df = df.iloc[:-1]

# Extract items and groups
items = []
groups = df['Group'].unique()
num_resources = len(constraint_row)

# Create a list of items with their resource usage and values
for _, row in df.iterrows():
    item = {
        'Group': row['Group'],
        'Item': row['Item'],
        'Value': row['Value'],
        'Resources': row[3:].values.astype(int)
    }
    items.append(item)

# Function to generate an initial population
def initial_population(items, pop_size):
    population = []
    for _ in range(pop_size):
        solution = initial_solution(items)
        population.append(solution)
    return population

# Function to evaluate the population
def evaluate_population(population, constraints):
    return [evaluate_solution(solution, constraints) for solution in population]

# Function to select parents for crossover
def select_parents(population, scores, num_parents):
    selected_indices = np.argsort(scores)[-num_parents:]
    return [population[i] for i in selected_indices]

# Function for crossover between two parents
def crossover(parent1, parent2):
    child = parent1.copy()
    for i in range(len(parent1)):
        if np.random.rand() < 0.5:
            child[i] = parent2[i]
    return child

# Function for mutation
def mutate(solution, items, mutation_rate=0.1):
    new_solution = solution.copy()
    if np.random.rand() < mutation_rate:
        group = np.random.choice(list(set(item['Group'] for item in solution)))
        group_items = [item for item in items if item['Group'] == group]
        if len(group_items) > 1:
            new_item = np.random.choice(group_items)
            idx = next(i for i, item in enumerate(new_solution) if item['Group'] == group)
            new_solution[idx] = new_item
    return new_solution

# Genetic Algorithm implementation
def genetic_algorithm(items, constraints, pop_size=50, num_generations=100, num_parents=10, mutation_rate=0.1):
    population = initial_population(items, pop_size)
    
    for _ in range(num_generations):
        scores = evaluate_population(population, constraints)
        parents = select_parents(population, scores, num_parents)
        new_population = []
        
        while len(new_population) < pop_size:
            parent1, parent2 = np.random.choice(len(parents), 2, replace=False)
            child = crossover(parents[parent1], parents[parent2])
            child = mutate(child, items, mutation_rate)
            new_population.append(child)
        
        population = new_population
    
    scores = evaluate_population(population, constraints)
    best_index = np.argmax(scores)
    best_solution = population[best_index]
    best_value = scores[best_index]
    
    return best_solution, best_value

# Run Genetic Algorithm
best_solution, best_value = genetic_algorithm(items, constraint_row)

# Output results
print("Best Solution:")
best_individual_value = -float('inf')  # Initialize with negative infinity
best_individual_item = None

for item in best_solution:
    print(f"Group: {item['Group']}, Item: {item['Item']}, Value: {item['Value']}")
    if item['Value'] > best_individual_value:
        best_individual_value = item['Value']
        best_individual_item = item

print("\nBest Individual Item Value:", best_individual_value)
if best_individual_item:
    print("Best Individual Item:", best_individual_item)
else:
    print("No valid individual item found.")
print("Resource Constraints:", constraint_row)


/tmp/ipykernel_23349/1287053409.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values


Best Solution:
Group: Group_87, Item: Item_1, Value: 164.0
Group: Group_90, Item: Item_3, Value: 156.0
Group: Group_36, Item: Item_17, Value: 185.0
Group: Group_37, Item: Item_6, Value: 163.0
Group: Group_23, Item: Item_17, Value: 184.0
Group: Group_91, Item: Item_25, Value: 192.0
Group: Group_56, Item: Item_21, Value: 192.0
Group: Group_98, Item: Item_11, Value: 192.0
Group: Group_53, Item: Item_5, Value: 173.0
Group: Group_6, Item: Item_18, Value: 178.0
Group: Group_92, Item: Item_2, Value: 188.0
Group: Group_48, Item: Item_5, Value: 182.0
Group: Group_58, Item: Item_21, Value: 181.0
Group: Group_96, Item: Item_12, Value: 173.0
Group: Group_81, Item: Item_11, Value: 198.0
Group: Group_66, Item: Item_2, Value: 186.0
Group: Group_43, Item: Item_12, Value: 191.0
Group: Group_69, Item: Item_23, Value: 171.0
Group: Group_17, Item: Item_17, Value: 196.0
Group: Group_28, Item: Item_12, Value: 179.0
Group: Group_46, Item: Item_17, Value: 187.0
Group: Group_27, Item: Item_24, Value: 166.0
Gro